In [235]:
import os
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import nltk 
import operator
import math
import scipy

In [187]:
global_POS = []
f = open('/home/rahmtin/Documents/Arxiv-Feynmann/Arxiv/Part Of Speech/POS.txt')
for line in f:
    global_POS.append(line[0:2])

global_POS = list(set(global_POS))


['PR', 'MD', 'VB', 'WD', 'RP', 'PD', 'NN', 'FW', 'CC', 'RB', 'CD', 'TO', 'UH', 'EX', 'JJ', 'IN', 'WP', 'DT', 'WR', 'PO', 'LS']


In [185]:
class Paper:
    def __init__(self, title):
         self.title = title
         self.authors = []
         self.experiences = []
         self.year = ''
         self.pos = []
         self.features = dict()
         self.tag = ""
         
    def to_string(self):
        return [self.title, self.year, self.authors, self.features]
    

f = open('/home/rahmtin/Documents/Arxiv-Feynmann/Arxiv/Part Of Speech/TitlesAndAuthorsAndMonth4.txt')
papers = []
paper = Paper('test')
counter = 0;
global_features = [':', '?' , 'MATH']
tag = ""
for line in f:
    try:
        text = nltk.word_tokenize(line)
    except UnicodeDecodeError:
        for line2 in f:
            if len(nltk.word_tokenize(line2)) == 0:
                break
        continue

    if len(text) == 0:
        papers.append(paper)
        counter = counter + 1
#        if counter >= 50:
#            break
        continue

    if text[0] == 'Tag':
        tag = text[2]
        
    if text[0] == 'Title':
        title = (' '.join(text[2:])).lower().decode('utf-8', 'ignore').encode('utf-8', 'ignore')
        number = 0
        tmp = ''
        for x in title:
            if x == '$':
                if number == 1:
                    tmp = tmp + " MATH "
                number = 1 - number
                continue
            if number > 0:
                continue
            tmp = tmp + x
        paper = Paper(' '.join(tmp.split()))
        paper.pos = nltk.pos_tag(nltk.word_tokenize(paper.title))
        if len(paper.pos) > 0:
            paper.pos[0] = (paper.pos[0][0], paper.pos[0][1][:2])
        paper.tag = tag
        for feature in global_features: 
            paper.features[feature] = 0
            if feature in paper.title:
                paper.features[feature] = 1
            
    
    if text[0] == 'Year':
        if text[2][0] == '9':
            text[2] = '19' + text[2]
        else:
            text[2] = '20' + text[2]
        paper.year = text[2]
    if text[0] == 'Author':
        author = ' '.join(text[2:]).lower()
        paper.authors.append(' '.join(author.split()))
    
papers.sort(key=operator.attrgetter('year'))
author_experience = dict()
for paper in papers:
    for author in paper.authors:
        if author not in author_experience:
            author_experience[author] = 0
        paper.experiences.append(author_experience[author])
        author_experience[author] = author_experience[author] + 1

#    print "---->", paper.authors, paper.year
#    print paper.experiences

In [214]:
import sys
def find_scores(paper_list):
    if(len(paper_list) < min_for_min_threshold):
        sys.exit()
    ret = dict()
    for feature in global_features:
        ret[feature] = 0
    for feature in global_POS:
        ret[feature] = 0
    for index in paper_list:
        paper = papers[index]
        for feature in global_features:
            ret[feature] = ret[feature] + paper.features[feature]
        for feature in global_POS:
            if len(paper.pos) > 0:
                ret[feature] = ret[feature] + int(paper.pos[0][1] == feature)
    return ret

In [246]:
author_papers = dict()

fight_count = 0
author_pairs = set()
regression_scores = dict()
regression_label = dict()
denom = dict()
num = dict()
ratio = dict()
first_pos_point = []

min_for_min_threshold = 1
max_for_min_threshold = 2

for feature in global_features + global_POS:
    denom[feature] = dict()
    num[feature] = dict()
    ratio[feature] = dict()
    for i in range (0, max_for_min_threshold + 1):
        for j in range (0, max_for_min_threshold + 1):
            num[feature][(i, j)] = 0
            denom[feature][(i, j)] = 0
    regression_scores[feature] = []
    regression_label[feature] = []
    
for index, paper in enumerate(papers):
    if len(paper.authors) == 2 and len(paper.pos) > 0:
        if tuple(paper.authors) not in author_pairs:
            older_index = younger_index = 0
            if paper.experiences[0] < paper.experiences[1]: 
                older_index = 1
            else:
                younger_index = 1

            if(min(paper.experiences) >= min_for_min_threshold and min(paper.experiences) <= max_for_min_threshold and
              paper.experiences[older_index] / paper.experiences[younger_index] > 2 and 
              paper.experiences[older_index] - paper.experiences[younger_index] > 10):
                #print paper.experiences, paper.authors, younger_index, older_index
                # we found a fight
                fight_count += 1
                older_papers = author_papers[paper.authors[older_index]]
                younger_papers = author_papers[paper.authors[younger_index]]
                older_scores = find_scores(older_papers[-min(paper.experiences):])
                younger_scores = find_scores(younger_papers[-min(paper.experiences):])
                for feature in global_features + global_POS:
                    younger_score = younger_scores[feature]
                    older_score = older_scores[feature]
                    denom[feature][(younger_score, older_score)] = denom[feature][(younger_score, older_score)] + 1

                    regression_scores[feature].append([younger_score, older_score])
                    if feature in global_features:
                        regression_label[feature].append(paper.features[feature])
                        if paper.features[feature] == 0:
                            continue
                        num[feature][(younger_score, older_score)] = num[feature][(younger_score, older_score)] + 1
                    if feature in global_POS:
                        regression_label[feature].append(int(paper.pos[0][1] == feature))
                        num[feature][(younger_score, older_score)] = num[feature][(younger_score, older_score)] + int(paper.pos[0][1] == feature)


                   
                
    for author1 in paper.authors:
        for author2 in paper.authors:
            author_pairs.add((author1, author2))
            author_pairs.add((author2, author1))
        if author1 not in author_papers:
            author_papers[author1] = []
        author1_papers = author_papers[author1]
#        while len(author1_papers) >= min_for_min_threshold:
#            author1_papers.pop(0)
        author1_papers.append(index)
        author_papers[author1] = author1_papers

print fight_count

for feature in global_features + global_POS:
    for i in range (0, min_for_min_threshold + 1):
        for j in range (0, min_for_min_threshold + 1):
            denom[feature][(i,j)] = max(1, denom[feature][(i,j)])
            ratio[feature][(i,j)] =  (num[feature][(i,j)] / float(denom[feature][(i,j)]))

5709


In [247]:
relevant_features = ['VB', 'NN', 'RB', 'JJ', 'DT']
for feature in relevant_features + global_features:
    for i in range (0, min(3, min_for_min_threshold + 1)):
        for j in range (0, min(3, min_for_min_threshold + 1)):
            if i < j: # i is younger, j is older
                #(younger less usage, older more usage) - (older less usage, younger more usage)
                print num[feature][(j,i)], denom[feature][(j,i)], ratio[feature][(j,i)]
                
                
                p_value = scipy.stats.binom_test(num[feature][(j,i)], denom[feature][(j,i)], ratio[feature][(i,j)], alternative='two-sided')
                
                mean1 = ratio[feature][(i,j)]
                var1 = mean1 * (1 - mean1)
                #print mean1, (1 - mean1), var1
                mean2 = ratio[feature][(j,i)]
                var2 = mean2 * (1 - mean2)
                #print mean2, (1 - mean2), var2
                sample_numbers = denom[feature][(i,j)] + denom[feature][(j,i)]
                CI = 1.96  * (var1 + var2) / (math.sqrt(denom[feature][(i,j)]) + math.sqrt(denom[feature][(j,i)]))
                print (feature, i, j,
                       str(ratio[feature][(i,j)] - ratio[feature][(j,i)]) + " +/- " + str(CI), p_value,
                       num[feature][(i,j)], denom[feature][(i,j)],
                       num[feature][(j,i)], denom[feature][(j,i)] )


41 468 0.0876068376068
('VB', 0, 1, '0.011796740922 +/- 0.007537977589', 0.43972312873229263, 50, 503, 41, 468)
393 1073 0.366262814539
('NN', 0, 1, '0.00893341780041 +/- 0.0133589145589', 0.57040853986788853, 478, 1274, 393, 1073)
5 113 0.0442477876106
('RB', 0, 1, '-0.0271537705166 +/- 0.00540033001648', 0.04519718785758408, 2, 117, 5, 113)
411 1153 0.35646140503
('JJ', 0, 1, '0.00762426218546 +/- 0.0131312871335', 0.60298242872773034, 442, 1214, 411, 1153)
104 660 0.157575757576
('DT', 0, 1, '0.0354622171078 +/- 0.011125285661', 0.020401051991723408, 122, 632, 104, 660)
73 532 0.137218045113
(':', 0, 1, '0.0396207655288 +/- 0.0107016073822', 0.016850085328458268, 113, 639, 73, 532)
5 87 0.0574712643678
('?', 0, 1, '0.0114942528736 +/- 0.0115446037035', 0.83354593039078695, 8, 116, 5, 87)
104 518 0.200772200772
('MATH', 0, 1, '0.0342101313833 +/- 0.0143252914221', 0.069637685614430345, 133, 566, 104, 518)


In [248]:
# from sklearn import linear_model
clf = linear_model.LinearRegression() # first one is younger, second one is older
#print regression_scores[':']
clf.fit(regression_scores[':'], regression_label[':'])
print(': Coefficients: \n', clf.coef_)
clf.fit(regression_scores['?'], regression_label['?'])
print('? Coefficients: \n', clf.coef_)
clf.fit(regression_scores['MATH'], regression_label['MATH'])
print('MATH Coefficients: \n', clf.coef_)

(': Coefficients: \n', array([ 0.04995714,  0.09220335]))
('? Coefficients: \n', array([ 0.0498383 ,  0.07224182]))
('MATH Coefficients: \n', array([ 0.1229862 ,  0.14732743]))


In [249]:

older_wins_count = 0                                                                           
younger_wins_count = 0
for x in first_pos_point:
    if x[0] < x[1]:
        older_wins_count += 1
    elif x[1] < x[0]:
        younger_wins_count += 1

print "younger, older wins: ", younger_wins, older_wins

younger, older wins:  781 761


1, 4
('VB', 0, 1, '0.0079744504095 +/- 0.0116881623418', 93, 906, 80, 845)
('NN', 0, 1, '0.00184038792014 +/- 0.0145825962205', 575, 1573, 495, 1361)
('RB', 0, 1, '-0.0103448275862 +/- 0.012117935718', 6, 261, 8, 240)
('JJ', 0, 1, '0.00555136363678 +/- 0.0144085151544', 538, 1517, 502, 1438)
('DT', 0, 1, '0.0227478439531 +/- 0.0128732941796', 174, 1049, 153, 1069)
(':', 0, 1, '0.0231641366866 +/- 0.0127965577153', 160, 1063, 115, 903)
('?', 0, 1, '0.0257936507937 +/- 0.0169115196492', 12, 216, 5, 168)
('MATH', 0, 1, '0.0193569643887 +/- 0.0150485142735', 171, 903, 144, 847)
1, 3

___________________________________________________________________
min_for_min_threshold = 1
max_for_min_threshold = 1

heat_map:
(':', 0, 1, 0.03537735849056603, 68, 424, 45, 360)
('?', 0, 1, 0.0, 2, 44, 2, 44)
('MATH', 0, 1, 0.033664270227091175, 100, 460, 70, 381)
Regression:
(': Coefficients: \n', array([ 0.03278064,  0.06894976]))
('? Coefficients: \n', array([ 0.02713494,  0.02713494]))
('MATH Coefficients: \n', array([ 0.1203563 ,  0.15234195]))
#########################
min_for_min_threshold = 1
max_for_min_threshold = 2
heat_map:
(':', 0, 1, 0.022896236944909526, 106, 678, 79, 592)
('?', 0, 1, 0.02524893314366998, 5, 76, 3, 74)
('MATH', 0, 1, 0.014001302842837127, 157, 754, 121, 623)
Regression:
(': Coefficients: \n', array([ 0.0400271 ,  0.06391486]))
('? Coefficients: \n', array([ 0.02350188,  0.04889978]))
('MATH Coefficients: \n', array([ 0.13469944,  0.14592531]))
##########################
min_for_min_threshold = 1
max_for_min_threshold = 3

heat_map:
(':', 0, 1, 0.008629538455948493, 123, 849, 106, 778)
('?', 0, 1, 0.038457083285068916, 7, 97, 3, 89)
('MATH', 0, 1, 0.007802046438410093, 200, 968, 167, 840)
Regression:
(': Coefficients: \n', array([ 0.04304085,  0.05219309]))
('? Coefficients: \n', array([ 0.01800377,  0.05685816]))
('MATH Coefficients: \n', array([ 0.14031934,  0.14578552]))
#########################
min_for_min_threshold = 1
max_for_min_threshold = 4
heat_map:
(':', 0, 1, 0.022896236944909526, 106, 678, 79, 592)
('?', 0, 1, 0.02524893314366998, 5, 76, 3, 74)
('MATH', 0, 1, 0.014001302842837127, 157, 754, 121, 623)
Regression:
(': Coefficients: \n', array([ 0.0400271 ,  0.06391486]))
('? Coefficients: \n', array([ 0.02350188,  0.04889978]))
('MATH Coefficients: \n', array([ 0.13469944,  0.14592531]))

#####################
min_for_min_threshold = 1
max_for_min_threshold = 5

heat_map
(':', 0, 1, 0.009307138038498308, 167, 1165, 139, 1037)
('?', 0, 1, 0.001914562642096447, 7, 137, 6, 122)
('MATH', 0, 1, -0.019669401499506994, 254, 1311, 245, 1148)
Regression:
(': Coefficients: \n', array([ 0.0400271 ,  0.06391486]))
('? Coefficients: \n', array([ 0.02350188,  0.04889978]))
('MATH Coefficients: \n', array([ 0.13469944,  0.14592531]))

#####################
min_for_min_threshold = 1
max_for_min_threshold = 6
heat_map:
(':', 0, 1, 0.010793864131159209, 180, 1268, 149, 1136)
('?', 0, 1, 0.008941272099166842, 8, 148, 6, 133)
('MATH', 0, 1, -0.02665396181551158, 273, 1448, 272, 1264)
Regression:
(': Coefficients: \n', array([ 0.0400271 ,  0.06391486]))
('? Coefficients: \n', array([ 0.02350188,  0.04889978]))
('MATH Coefficients: \n', array([ 0.13469944,  0.14592531]))
#######################
min_for_min_threshold = 1
max_for_min_threshold = 7
heat_map:
(':', 0, 1, 0.00866123171914629, 191, 1363, 160, 1217)
('?', 0, 1, 0.007148997782905751, 8, 159, 6, 139)
('MATH', 0, 1, -0.02596477423347099, 296, 1556, 291, 1346)
Regression:
(': Coefficients: \n', array([ 0.0400271 ,  0.06391486]))
('? Coefficients: \n', array([ 0.02350188,  0.04889978]))
('MATH Coefficients: \n', array([ 0.13469944,  0.14592531]))
#######################
min_for_min_threshold = 1
max_for_min_threshold = 8

(':', 0, 1, 0.003390499927896029, 199, 1444, 173, 1287)
('?', 0, 1, 0.00784313725490196, 8, 170, 6, 153)
('MATH', 0, 1, -0.022421182106557913, 307, 1626, 301, 1425)
Regression:
(': Coefficients: \n', array([ 0.04378242,  0.04742679]))
('? Coefficients: \n', array([ 0.02514499,  0.03326163]))
('MATH Coefficients: \n', array([ 0.14442149,  0.12010365]))